## Etape 1 : Scraping des résultats depuis bases.athle

Objectifs :
+ créer un pipeline pour récupérer les données d'une compétition d'athlétisme (course sur route, trail, cross, ...)
+ retourner ces données sous la forme d'une table relationnelle
+ par la suite, exploiter cette table pour créer des statistiques et visualisations sur les compétitions

Format d'une URL de bases.athle :

+ ? avant la requête
+ & pour ajouter des paramètres à la recherche 

bases.athle.fr/asp.net/liste.aspx**?frmbase=resultats**&**frmmode=1**&**frmespace=0**&**frmcompetition=**274211

Liste des paramètres pour une recherche par athlète :
+ frmsaison
+ frmclub
+ frmnom
+ frmprenom
+ frmsexe
+ frmlicence
+ frmdepartement
+ frmligue
+ frmcomprch

### Installation de la libraire lxml

In [1]:
!pip install lxml

You should consider upgrading via the 'c:\users\trist\anaconda3\python.exe -m pip install --upgrade pip' command.


### Import des librairies requises

In [2]:
import lxml
import urllib
import bs4
import pandas
import re
from urllib import request

In [3]:
#Exemple de résultats de compétition
url="https://bases.athle.fr/asp.net/liste.aspx?frmbase=resultats&frmmode=1&frmespace=0&frmcompetition=253657"

In [4]:
request_text = request.urlopen(url).read()

In [5]:
page = bs4.BeautifulSoup(request_text, "lxml")

## Date, nom, lieu, ligue, label et département de la compétition

In [6]:
header = page.find('div', {'class' : "mainheaders"})

In [7]:
header

<div class="mainheaders">06/11/22 - 10 km de Riorges<br/><span style="font-size:13px">Riorges - ARA - 042 - Label Régional</span></div>

#### Nom de la compétition

In [8]:
re_nom = re.compile("(?<=\- )(.*?)(?=\<)")
nom = re_nom.findall(str(header))
print(nom[0])

10 km de Riorges


#### Lieu de la compétition

In [9]:
re_lieu = re.compile("(?<=\>)(\D*?)(?=\ -)")
lieu = re_lieu.findall(str(header))
print(lieu[0])

Riorges


#### Date de la compétition

In [10]:
re_date = re.compile("[0-9]{2}/[0-9]{2}/[0-9]{2}")
date = re_date.findall(str(header))
print(date[0])

06/11/22


#### Ligue de la compétition

In [17]:
soustitre = nom[1]
re_ligue = re.compile("[A-Z]{3}")
ligue = re_ligue.findall(soustitre)
print(ligue[0])

ARA


#### Département de la compétition

In [18]:
re_dept = re.compile("[0-9]{3}")
dept = re_dept.findall(soustitre)
print(dept[0])

042


#### Label de la compétition (en cours)

In [21]:
soustitre

'ARA - 042 - Label Régional'

In [23]:
re_label = re.compile("(?<=Label).*(.*$)")
label = re_label.findall(soustitre)
print(label)

['']


In [24]:
print(page.find("title"))

<title>les Résultats des Compétitions</title>


In [25]:
print(page.find("mainheaders"))

None


In [26]:
print("Il y a", len(page.findAll("tr")), "éléments dans la page qui sont des <tr>")

Il y a 257 éléments dans la page qui sont des <tr>


In [27]:
print(page.find('a', {'title' : 'Records principaux pour cette épreuve'}))

<a class="white" href="/asp.net/liste.aspx?frmbase=records&amp;frmmode=2&amp;frmespace=0&amp;frmcompetition=253657&amp;frmepreuve=261&amp;frmsexe=M&amp;frmserie=10 Km Route TC" title="Records principaux pour cette épreuve">10 Km Route TC</a>


In [28]:
print(page.find('tr'))

<tr> <td style="width:2px"><div class="separator"></div></td> <td class="barButtons"><span class="barButtonsOff" onclick="history.back()" onmouseout="this.className='barButtonsOff'" onmouseover="this.className='barButtonsOn'" style="width:25px;background-image:url('/images/v3/bar.back.gif')" title="Retour"> </span></td> <td style="width:2px"><div class="separator"></div></td><td class="barInputs"> <select class="barSelect" onchange="document.location='/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmposition=' + this.options[this.selectedIndex].value"><option selected="selected" value="0">Page &gt; 001/002 &lt;</option><option value="1">Page &gt; 002/002 &lt;</option></select>  <select class="barSelect" onchange="document.location='/asp.net/liste.aspx?frmbase=resultats&amp;frmmode=1&amp;frmespace=0&amp;frmcompetition=253657&amp;frmtri=' + this.options[this.selectedIndex].value"><option selected="selected" value="0">Tri &gt; Cat./Sex./